In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from d2l import torch as d2l

In [2]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
    
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return F.relu(Y + X)

In [10]:
blk = Residual(3, 3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
print(Y.shape)
for layer in blk.children():
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

torch.Size([4, 3, 6, 6])
Conv2d output shape:	 torch.Size([4, 3, 6, 6])
Conv2d output shape:	 torch.Size([4, 3, 6, 6])
BatchNorm2d output shape:	 torch.Size([4, 3, 6, 6])
BatchNorm2d output shape:	 torch.Size([4, 3, 6, 6])


In [12]:
blk = Residual(3, 6, use_1x1conv=True, strides=2)
X = torch.rand(4, 3, 6, 6)
print(blk(X).shape)
for layer in blk.children():
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

torch.Size([4, 6, 3, 3])
Conv2d output shape:	 torch.Size([4, 6, 3, 3])
Conv2d output shape:	 torch.Size([4, 6, 3, 3])


RuntimeError: Given groups=1, weight of size [6, 3, 1, 1], expected input[4, 6, 3, 3] to have 3 channels, but got 6 channels instead